In [1]:
# importing dependencies 

import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import tensorflow as tf 
from tensorflow.python.framework import ops

%matplotlib inline

## Visualize dataframe

In [2]:
# importing data using pandas 
dfz = pd.read_excel('BTC daily_database.xlsx', sheet_name=0)
# testing the input is correct or not
print(dfz.head())

        Date  Price       RSI  Stedev from 20 MA  MACD n.f  50MA_N.F.  \
0 2011-12-16   3.25  0.514151           0.591527  1.594689   0.500194   
1 2011-12-18   3.25  0.443243           0.516482  1.489778   0.571086   
2 2011-12-19   3.50  0.511962           1.311777  1.473728   1.266368   
3 2011-12-20   4.75  0.822695           3.461711  1.840524   3.831441   
4 2011-12-21   4.38  0.753247           2.180044  1.969366   2.760808   

   20D PMO  35D PMO        ADX  actions  y-hat  Unnamed: 11  Unnamed: 12  
0    3.270     0.59  69.772809      NaN      2            1  2157.000000  
1    3.690     0.46  70.038037      NaN      2            1     0.912437  
2    4.275     0.67  70.475308      NaN      2            1          NaN  
3    5.140     1.55  68.503186      NaN      2            1          NaN  
4    5.910     1.28  66.671929      NaN      2            1          NaN  


## Randomize sampling

In [3]:
# shuffler() will read the data and randomly organizes it 
def shuffler(filename):
  # Read an Excel table into a pandas DataFrame
  # takes the first sheet and header
    
  dfs = pd.read_excel(filename, sheet_name=0, header=0)
  # return the pandas dataframe
  # np.random.permutation shuffles the data points 
  # dfs will clean the data by placing NaN at x-features that have no data 
  return dfs.reindex(np.random.permutation(dfs.index))

In [4]:
# randomizes the data and saves it in df
df = shuffler('BTC daily_database.xlsx')

# prints the first 5 rows of the data table
print(df.head())

           Date    Price       RSI  Stedev from 20 MA  MACD n.f  50MA_N.F.  \
98   2012-03-24     4.72  0.441718          -1.372787 -0.172724  -0.864722   
828  2014-03-24   558.22  0.164125          -1.995781 -0.159128  -1.200441   
2137 2017-10-26  5741.35  0.684951           0.618055  0.796806   1.450945   
206  2012-07-10     6.95  0.789916           2.373414  0.376840   1.602152   
176  2012-06-10     5.54  0.859375           1.878567  2.907242   2.878015   

      20D PMO  35D PMO        ADX  actions  y-hat  Unnamed: 11  Unnamed: 12  
98     -1.690     0.11  24.690947      NaN      2            1          NaN  
828    -2.995    -0.19  41.472985      NaN      2            1          NaN  
2137    8.795     1.21  22.638714      NaN      2            1          NaN  
206     7.555     0.85  14.373037      NaN      2            1          NaN  
176     0.780     0.27  26.943206      NaN      2            1          NaN  


## Input dataframe

In [5]:
# converts to numpy array using as_matrix()
# use iloc() to get location 
inputX = df.iloc[:,2:9 ].as_matrix()

# transposes inputX
inputX = inputX.T

# converts to numpy array using as_matrix()
# use iloc() to get location 
inputY = df.iloc[:, 10:11].as_matrix()

# transposes inputY
inputY = inputY.T

print("inputX shape: " + str(inputX.shape))
print("number of training samples = "+ str(inputX.shape[1]))
print("number of variables = " + str(inputX.shape[0]))
print(inputX)
print(inputY.shape)

inputX shape: (7, 2363)
number of training samples = 2363
number of variables = 7
[[  4.41717791e-01   1.64125335e-01   6.84951073e-01 ...,   8.15602837e-01
    8.48794226e-01   5.94053119e-01]
 [ -1.37278699e+00  -1.99578130e+00   6.18054525e-01 ...,   2.67979882e+00
    2.29682429e+00   9.77566896e-01]
 [ -1.72723616e-01  -1.59127857e-01   7.96805614e-01 ...,   3.25286892e-01
    3.12572020e+00   1.16546495e+00]
 ..., 
 [ -1.69000000e+00  -2.99500000e+00   8.79500000e+00 ...,  -2.98500000e+00
    1.22500000e+00   1.23300000e+01]
 [  1.10000000e-01  -1.90000000e-01   1.21000000e+00 ...,  -5.00000000e-02
    3.30000000e-01   1.08000000e+00]
 [  2.46909473e+01   4.14729851e+01   2.26387138e+01 ...,   5.60678609e+01
    1.75089142e+01   1.32235335e+01]]
(1, 2363)


In [6]:
inputY

array([[2, 2, 2, ..., 2, 3, 2]])

## Seperating to train and test datasets

In [7]:
# selects trainning data 
trainX = inputX[:, : -470]
trainY = inputY[:, : -470]

# selects test data 
testX = inputX[:, -470: ]
testY = inputY[:, -470: ]

# verifies the shapes 
print("trainX's shape: " + str(trainX.shape))
print("testX's shape: " + str(testX.shape))
print("trainX's shape: " + str(trainY.shape))
print("testY's shape: " + str(testY.shape))


trainX's shape: (7, 1893)
testX's shape: (7, 470)
trainX's shape: (1, 1893)
testY's shape: (1, 470)


## Layer size inputs 

In [8]:
# size of input layer
n_x = inputX.shape[0]

# amount of hidden layers
n_h = 10

# output layer 
n_y = inputY.shape[0]

print("n_x's shape = " + str(n_x))
print("first layer(n_h)'s shape = " + str(n_h))
print("n_y's output = " + str(n_y))

n_x's shape = 7
first layer(n_h)'s shape = 10
n_y's output = 1


## Create placeholder


In [9]:
# Creates the placeholders for the tensorflow session.
def create_placeholders(n_x, n_y):
    
    # creates placholder for x and y that batch size can be any size
    X = tf.placeholder(tf.float32, shape = [n_x,None])
    Y = tf.placeholder(tf.float32, shape = [n_y, 1, None])  
    return X, Y

In [10]:
X, Y = create_placeholders(n_x,n_y)
print ("X = " + str(X))
print(X.shape)
print ("Y = " + str(Y))

X = Tensor("Placeholder:0", shape=(7, ?), dtype=float32)
(7, ?)
Y = Tensor("Placeholder_1:0", shape=(1, 1, ?), dtype=float32)


## Initialize parameters

In [11]:
#  Initializes weight parameters to build a neural network with tensorflow  
def initialize_parameters(n_x, n_h):
    #tf.set_random_seed()
    
    # Layer 1 (Input Layer)
    W1 = tf.get_variable("W1", [n_h,n_x], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    b1 = tf.get_variable("b1", [n_h,1], initializer = tf.zeros_initializer())
    
    # Layer 2 (10 neurons)
    W2 = tf.get_variable("W2", [10, n_h], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    b2 = tf.get_variable("b2", [10, 1], initializer = tf.zeros_initializer())
    
    # Layer 3 (10 neurons)
    W3 = tf.get_variable("W3", [25, 10], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    b3 = tf.get_variable("b3", [25, 1], initializer = tf.zeros_initializer())
    
    # Layer 4 (10 neurons)
    W4 = tf.get_variable("W4", [10, 25], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    b4 = tf.get_variable("b4", [10,1], initializer = tf.zeros_initializer())
    
    # Layer 5 (10 neurons)
    W5 = tf.get_variable("W5", [5, 10], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    b5 = tf.get_variable("b5", [5,1], initializer = tf.zeros_initializer())
    
    #W6 = tf.get_variable("W6", [5, 10], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    #b6 = tf.get_variable("b6", [5,1], initializer = tf.zeros_initializer())
    #W7 = tf.get_variable("W7", [5, 10], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    #b7 = tf.get_variable("b7", [5,1], initializer = tf.zeros_initializer())
    
    # saves weights and bias for layers as a dictionary 
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3,
                  "W4": W4,
                  "b4": b4,
                  "W5": W5,
                  "b5": b5}
                  #"W6": W6,
                  #"b6": b6,
                  #"W7": W7,
                  #"b7": b7}

    return parameters

In [12]:
# Checking the shape of the weights and bias 
tf.reset_default_graph()

with tf.Session() as sess:
    parameters = initialize_parameters(n_x, n_h)
    print("W1 = " + str(parameters["W1"]))
    print("b1 = " + str(parameters["b1"]))
    print("W2 = " + str(parameters["W2"]))
    print("b2 = " + str(parameters["b2"]))
    print("W3 = " + str(parameters["W3"]))
    print("b3 = " + str(parameters["b3"]))
    print("W4 = " + str(parameters["W4"]))
    print("b4 = " + str(parameters["b4"]))
    print("W5 = " + str(parameters["W5"]))
    print("b5 = " + str(parameters["b5"]))
    #print("W6 = " + str(parameters["W6"]))
    #print("b6 = " + str(parameters["b6"]))
    #print("W7 = " + str(parameters["W7"]))
    #print("b7 = " + str(parameters["b7"]))

W1 = <tf.Variable 'W1:0' shape=(10, 7) dtype=float32_ref>
b1 = <tf.Variable 'b1:0' shape=(10, 1) dtype=float32_ref>
W2 = <tf.Variable 'W2:0' shape=(10, 10) dtype=float32_ref>
b2 = <tf.Variable 'b2:0' shape=(10, 1) dtype=float32_ref>
W3 = <tf.Variable 'W3:0' shape=(25, 10) dtype=float32_ref>
b3 = <tf.Variable 'b3:0' shape=(25, 1) dtype=float32_ref>
W4 = <tf.Variable 'W4:0' shape=(10, 25) dtype=float32_ref>
b4 = <tf.Variable 'b4:0' shape=(10, 1) dtype=float32_ref>
W5 = <tf.Variable 'W5:0' shape=(5, 10) dtype=float32_ref>
b5 = <tf.Variable 'b5:0' shape=(5, 1) dtype=float32_ref>


## One hot matrix

In [13]:
def one_hot_matrix(labels, C):
    """
    Creates a matrix where the i-th row corresponds to the ith class number and the jth column
                     corresponds to the jth training example. So if example j had a label i. Then entry (i,j) 
                     will be 1. 
                     
    Arguments:
    labels -- vector containing the labels 
    C -- number of classes, the depth of the one hot dimension
    
    Returns: 
    one_hot -- one hot matrix
    """
    # Create a tf.constant equal to C (depth), name it 'C'. (approx. 1 line)
    C = tf.constant(C, name = "C")
    
    # Use tf.one_hot, be careful with the axis (approx. 1 line)
    one_hot_matrix = tf.one_hot(indices = labels, depth = C,axis  =0)
    
    # Create the session (approx. 1 line)
    sess = tf.Session()
    
    # Run the session (approx. 1 line)
    one_hot = sess.run(one_hot_matrix)
    
    # Close the session (approx. 1 line). See method 1 above.
    sess.close()
    
    return one_hot

In [14]:
# Testing one_hot
labels = np.array([1,2,3,0,2,1])
one_hot = one_hot_matrix(labels, C = 4)
print ("one_hot = " + str(one_hot))

one_hot = [[ 0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.  0.]
 [ 0.  0.  1.  0.  0.  0.]]


## Convert Y-hat to softmax matrix

In [15]:
# use one_hot on trainY and testY
trainY = one_hot_matrix(trainY, C = 5)
testY = one_hot_matrix(testY, C = 5)

# bug ? 
print("Y = ")
print(trainY)

print("trainY-shape = " + str(trainY.shape))
print("testY-shape = " + str(testY.shape))

Y = 
[[[ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 1.  1.  1. ...,  1.  1.  1.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]]]
trainY-shape = (5, 1, 1893)
testY-shape = (5, 1, 470)


## Forward Propagation

In [16]:
def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3", "W4", "b4"
                  the shapes are given in initialize_parameters

    Returns:
    Z4 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    W4 = parameters['W4']
    b4 = parameters['b4']
    W5 = parameters['W5']
    b5 = parameters['b5'] 
    #W6 = parameters['W6']
    #b6 = parameters['b6'] 
    #W7 = parameters['W7']
    #b7 = parameters['b7'] 
    
    # Input Layer (Sigmoid)
    Z1 = tf.add(tf.matmul(W1,X) , b1 )                              
    A1 = tf.nn.tanh(Z1)
    
    # 1st Hidden Layer (Sigmoid)
    Z2 = tf.add(tf.matmul(W2,A1), b2 )                               
    A2 = tf.nn.sigmoid(Z2)
    
    # 2nd Hidden Layer (Selu)
    Z3 = tf.add(tf.matmul(W3,A2), b3)
    A3 = tf.nn.selu(Z3) 
    
    # 3rd Hidden Layer (Selu)
    Z4 = tf.add(tf.matmul(W4,A3), b4)                                
    A4 = tf.nn.selu(Z4)             
    
    # Output Layer
    Z5 = tf.add(tf.matmul(W5,A4), b5)                                 
    #A5 = tf.nn.tanh(Z5)                                              
    #Z6 = tf.add(tf.matmul(W6,A5), b6)                                 
    #A6 = tf.nn.relu(Z6)                                              
    #Z7 = tf.add(tf.matmul(W7,A6), b7)                                
    return Z5
    

## Compute Cost

In [17]:
def compute_cost(Z, Y):
    """
    Computes the cost
    
    Arguments:
    Z4 -- output of forward propagation (output of the last LINEAR unit), of shape (6, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z4
    
    Returns:
    cost - Tensor of the cost function
    """
    
    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z)
    labels = tf.transpose(Y)
    
    #compute cost
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
    
    return cost

In [18]:
tf.reset_default_graph()

with tf.Session() as sess:
    X, Y = create_placeholders(4, 1)
    parameters = initialize_parameters(4,1)
    Z1 = forward_propagation(X, parameters)
    cost = compute_cost(Z1, Y)
    print("cost = " + str(cost))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

cost = Tensor("Mean:0", shape=(), dtype=float32)


## Creating Mini-batches

In [19]:

def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)

    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (1, number of examples)
    mini_batch_size -- size of the mini-batches, integer

    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """

    np.random.seed(seed)            # To make your "random" minibatches the same as ours
    m = X.shape[1]                  # number of training examples
    mini_batches = []
    
    # Step 1: Shuffle (X, Y)
    #permutation = list(np.random.permutation(m))
    #shuffled_X = X[:, permutation]
    #shuffled_Y = Y[:, 1, permutation]
    
    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X = X[:, k*mini_batch_size : (k+1)*mini_batch_size]
        mini_batch_Y = Y[:,:, k*mini_batch_size : (k+1)*mini_batch_size]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X = X[:, num_complete_minibatches*mini_batch_size : m]
        mini_batch_Y = Y[:,:, num_complete_minibatches*mini_batch_size : m]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    return mini_batches

In [20]:
mini_batches = random_mini_batches(trainX, trainY, 64, seed=0)

print ("shape of the 1st mini_batch_X: " + str(mini_batches[0][0].shape))
print ("shape of the 2nd mini_batch_X: " + str(mini_batches[1][0].shape))
print ("shape of the 3rd mini_batch_X: " + str(mini_batches[2][0].shape))
print ("shape of the last mini_batch_X: " + str(mini_batches[-1][0].shape))
print ("shape of the 1st mini_batch_Y: " + str(mini_batches[0][1].shape))
print ("shape of the 2nd mini_batch_Y: " + str(mini_batches[1][1].shape)) 
print ("shape of the 3rd mini_batch_Y: " + str(mini_batches[2][1].shape))
print ("shape of the last mini_batch_Y: " + str(mini_batches[-1][1].shape))
#print ("mini batch sanity check: ")
#mini_batches[0][1]


shape of the 1st mini_batch_X: (7, 64)
shape of the 2nd mini_batch_X: (7, 64)
shape of the 3rd mini_batch_X: (7, 64)
shape of the last mini_batch_X: (7, 37)
shape of the 1st mini_batch_Y: (5, 1, 64)
shape of the 2nd mini_batch_Y: (5, 1, 64)
shape of the 3rd mini_batch_Y: (5, 1, 64)
shape of the last mini_batch_Y: (5, 1, 37)


## Building the model 

In [21]:
def model(trainX, trainY, testX, testY, learning_rate = 0.0013,
          num_epochs = 2200, minibatch_size = 64, print_cost = True):
    """
    Implements a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SOFTMAX.
    
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = trainX.shape                                # (n_x: input size, m : number of examples in the train set)
    n_y = trainY.shape[0]                                  # n_y : output size
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of shape (n_x, n_y)
    X, Y = create_placeholders(n_x, n_y)

    # Initialize parameters
    parameters = initialize_parameters(n_x, n_h)

    # Forward propagation: Build the forward propagation in the tensorflow graph
    Z = forward_propagation(X, parameters)
    
    
    # Cost function: Add cost function to tensorflow graph
    cost = compute_cost(Z,Y)

    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

    # Initialize all the variables
    init = tf.global_variables_initializer()

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:

        # Run the initialization
        sess.run(init)

        # Do the training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(trainX, trainY, minibatch_size, seed)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch

                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})

                epoch_cost += minibatch_cost / num_minibatches

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)

        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(Z), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print ("Train Accuracy:", accuracy.eval({X: trainX, Y: trainY}))
        print ("Test Accuracy:", accuracy.eval({X: testX, Y: testY}))
        return parameters

In [ ]:
# run the model 
parameters = model(trainX, trainY, testX,testY)

Cost after epoch 0: 0.653183
Cost after epoch 100: 0.276426
Cost after epoch 200: 0.260272
Cost after epoch 300: 0.253854
Cost after epoch 400: 0.249470
Cost after epoch 500: 0.246521
Cost after epoch 600: 0.242008
